In [121]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import KFold

from lightgbm_model import MyLightGBM
from catboost_model import MyCatboost
%run lightgbm_model.py
%run catboost_model.py

In [122]:
X = datasets.fetch_covtype().data[:3000]
y = datasets.fetch_covtype().target[:3000]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print(X_train.shape, y_test.shape)
print(np.unique(y_train))  # 7分类任务

(2700, 54) (300,)
[1 2 3 4 5 6 7]


In [123]:
enc = OrdinalEncoder()
y_train_enc = enc.fit_transform(y_train.reshape(-1, 1)).reshape(-1, )
y_test_enc = enc.transform(y_test.reshape(-1, 1)).reshape(-1, )
print(np.unique(y_train_enc))
print(y_train_enc.shape)

[0. 1. 2. 3. 4. 5. 6.]
(2700,)


In [124]:
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
lgb_params = {"objective": "multiclass",
              "num_class": 7,
              "verbose": -4, "metric": ("multi_logloss",)}

cat_params = {"loss_function": "MultiClass",
              "eval_metric": "MultiClass",
              "allow_writing_files": False,
              "verbose": True,
              "thread_count": -1,
              "use_best_model": True
              }

### lightgbm测试

In [ ]:

lgb_train_pre, lgb_test_pre, lgb_model_list = MyLightGBM(X_train_data=X_train,
                                                         y_train_data=y_train_enc,
                                                         X_test_data=X_test,
                                                         kfold=kfold,
                                                         params=lgb_params,
                                                         verbose_eval=100,
                                                         early_stopping_rounds=100,
                                                         feval=None, fweight=None, categorical_feature="auto")

In [126]:
print(lgb_train_pre.shape)
print(lgb_train_pre.shape)
print(lgb_model_list)

(2700, 7)
(2700, 7)
[<lightgbm.basic.Booster object at 0x0000022884F0F040>, <lightgbm.basic.Booster object at 0x00000228852FD8E0>, <lightgbm.basic.Booster object at 0x0000022885316460>, <lightgbm.basic.Booster object at 0x00000228849A3820>, <lightgbm.basic.Booster object at 0x00000228849A3760>]


### catboost测试

In [ ]:
cat_train_pre, cat_test_pre, cat_model_list = MyCatboost(X_train_data=X_train,
                                                         y_train_data=y_train_enc,
                                                         X_test_data=X_test,
                                                         kfold=kfold,
                                                         params=cat_params,
                                                         num_class=7,
                                                         early_stopping_rounds=100,
                                                         verbose_eval=200,
                                                         fweight=None)

In [129]:
print(cat_train_pre.shape)
print(cat_train_pre.shape)
print(cat_model_list)

(2700, 7)
(2700, 7)
[<catboost.core.CatBoost object at 0x00000228852E5B50>, <catboost.core.CatBoost object at 0x0000022885316760>, <catboost.core.CatBoost object at 0x0000022885313820>, <catboost.core.CatBoost object at 0x0000022885313520>, <catboost.core.CatBoost object at 0x0000022885313FA0>]
